In [47]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings(action='ignore')


# Bar chart will generate the whole countries' trade amount in the designated year.
# Input Values: year -> up to downloaded dataset, trade -> 1: import, 2: export
def bar_chart(year, trade): 
    total_path = './data/total/'
    total_file = str(year) + '_' + 'total' + ".csv"
    file_check = os.path.exists(total_path + total_file)
    
    # Check the existence of 'year_total.csv' file.
    if not file_check:
        print('There is no ' + total_file + ' in the folder')
    
    # Remove Asean & EU-28 data
    df = pd.read_csv(total_path + total_file)
    no_asean = df['Reporter'] != 'ASEAN' # code = 975
    no_eu = df['Reporter'] != 'EU-28' # code = 97
    no_other_asia = df['Reporter'] != 'Other Asia, nes' # code = 490
    df = df[no_asean & no_eu & no_other_asia] # Eliminate the reporters, such as Asean, EU, Other Asia.
    
    if trade == 1:
        df_import = df[df['Trade Flow Code']==1]
        df_import_qt = pd.DataFrame(df_import['Netweight (kg)'].groupby(df_import['Reporter']).sum())
        df_import_qt.sort_values(by='Netweight (kg)', ascending=False, inplace=True)
        df_import_qt.reset_index(inplace=True)

        plt.figure(figsize=(20,20))
        plt.barh(df_import_qt['Reporter'], df_import_qt['Netweight (kg)'])
        plt.margins(y=0.01)
        plt.xlabel('Quantity (kg)', fontsize=20)
        plt.ylabel('Countries', fontsize=10)
        plt.title('{} Plastic Waste Import Trade Amount By Countries'.format(year), fontsize=20)
        plt.savefig('{}_Bar_Import Trade Amount.png'.format(year), bbox_inches='tight')
        plt.show()
        
    elif trade == 2:
        df_export = df[df['Trade Flow Code']==2]
        df_export_qt = pd.DataFrame(df_export['Netweight (kg)'].groupby(df_export['Reporter']).sum())
        df_export_qt.sort_values(by='Netweight (kg)', ascending=False, inplace=True)
        df_export_qt.reset_index(inplace=True)
        
        plt.figure(figsize=(20,20))
        plt.barh(df_export_qt['Reporter'], df_export_qt['Netweight (kg)'])
        plt.margins(y=0.01)
        plt.xlabel('Quantity (kg)', fontsize=20)
        plt.ylabel('Countries', fontsize=10)
        plt.title('{} Plastic Waste Export Trade Amount By Countries'.format(year),fontsize=20)
        plt.savefig('{}_Bar_Export Trade Amount.png'.format(year), bbox_inches='tight')
        plt.show()
    
    else:
        print('Select trade number: Import=1, Export=2')
        
    return


# Pie chart will show the high ranked countries that take in high amount of trading.
# According to the input rank value you can get different pie chart.
# Input Values: year -> up to downloaded dataset, trade -> 1: import, 2: export, rank: 0~100. 
# If you give 100 for the rank input, you will see whole countries. If you give 0, you will see just others with 100% value.
def pie_chart(year, trade, rank):
    rank = rank / 100
    total_path = './data/total/'
    total_file = str(year) + '_' + 'total' + ".csv"
    file_check = os.path.exists(total_path + total_file)

    if not file_check:
        print('There is no ' + total_file + ' in the folder')
    
    # Remove Asean & EU-28 data
    df = pd.read_csv(total_path + total_file)
    no_asean = df['Reporter'] != 'ASEAN' # code = 975
    no_eu = df['Reporter'] != 'EU-28' # code = 97
    no_other_asia = df['Reporter'] != 'Other Asia, nes' # code = 490
    df = df[no_asean & no_eu & no_other_asia]
    
    if trade == 1:
        df_import = df[df['Trade Flow Code']==1]
        df_import_qt = pd.DataFrame(df_import['Netweight (kg)'].groupby(df_import['Reporter']).sum())
        df_import_qt.sort_values(by='Netweight (kg)', ascending=False, inplace=True)
        df_import_qt.reset_index(inplace=True)
        df_import_qt['Percentage'] = df_import_qt['Netweight (kg)']/df_import_qt['Netweight (kg)'].sum()
        
        # This code is needed for the rank input
        count = 0
        percent = 0
        
        for i in df_import_qt['Percentage']:
            percent = percent + i
            
            if percent >= rank: # If the sum of percentage is higher than input rank value, Stop and return the value.
                count = count + 1
                break
            
            else: # # If the sum of percentage is lower than input rank value, Keep summing until meet the input value.
                count += 1
        
        df_import_top = df_import_qt[0:count]
        df_import_top.reset_index(inplace=True)
        df_import_top.drop('index', axis=1, inplace=True)  

        others_pc = df_import_qt['Percentage'][count:].sum()
        others_nw = df_import_qt['Netweight (kg)'][count:].sum()
        new_row = {'Reporter':'Others', 'Netweight (kg)':others_nw, 'Percentage':others_pc}
        df_import_top = df_import_top.append(new_row, ignore_index=True)
        
        labels = df_import_top['Reporter']
        sizes = df_import_top['Percentage']
        
        # This code is needed to get a neat and clean pie chart. This will divide the pies.
        explode = list()
        
        for i in range(count+1): # '+1' is necessary because when you divide the pie, total sides of pie is always 1 more than the sum of pies. 
            explode.append(0.05)

        plt.figure(figsize=(9,9))
        plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, pctdistance=0.85, explode = explode)
        centre_circle = plt.Circle((0,0),0.60,fc='white')
        fig = plt.gcf()
        fig.gca().add_artist(centre_circle)

        plt.title('{} Plastic Waste Import Trade Percentage By Countries'.format(year),fontsize=20)
        plt.tight_layout()
        plt.savefig('{}_Pie_Import Pecentage.png'.format(year), bbox_inches='tight')
        plt.show()
    
    elif trade == 2:
        df_export = df[df['Trade Flow Code']==2]
        df_export_qt = pd.DataFrame(df_export['Netweight (kg)'].groupby(df_export['Reporter']).sum())
        df_export_qt.sort_values(by='Netweight (kg)', ascending=False, inplace=True)
        df_export_qt.reset_index(inplace=True)
        df_export_qt['Percentage'] = df_export_qt['Netweight (kg)']/df_export_qt['Netweight (kg)'].sum()
        
        count = 0
        percent = 0
        
        for i in df_export_qt['Percentage']:
            percent = percent + i
            
            if percent >= rank:
                count = count + 1
                break
            
            else:
                count += 1
        
        df_export_top = df_export_qt[0:count]
        df_export_top.reset_index(inplace=True)
        df_export_top.drop('index', axis=1, inplace=True)  

        others_pc = df_export_qt['Percentage'][count:].sum()
        others_nw = df_export_qt['Netweight (kg)'][count:].sum()
        new_row = {'Reporter':'Others', 'Netweight (kg)':others_nw, 'Percentage':others_pc}
        df_export_top = df_export_top.append(new_row, ignore_index=True)
        
        labels = df_export_top['Reporter']
        sizes = df_export_top['Percentage']
        
        explode = list()
        
        for i in range(count+1):
            explode.append(0.05)

        plt.figure(figsize=(9,9))
        plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, pctdistance=0.85, explode = explode)
        centre_circle = plt.Circle((0,0),0.60,fc='white')
        fig = plt.gcf()
        fig.gca().add_artist(centre_circle)

        plt.title('{} Plastic Waste Export Trade Percentage By Countries'.format(year),fontsize=20)
        plt.tight_layout()
        plt.savefig('{}_Pie_Export Pecentage.png'.format(year), bbox_inches='tight')
        plt.show()
    
    else:
        print('Select trade number: Import=1, Export=2')
        
    return


# Line chart will give you the changes of trading amount of the countris accross the years.
# Input Values: trade -> 1: import, 2: export, Reporter -> country name, multiple reporter should be a list form.
def line_chart(trade, countries):
    
    # Making whole total file
    total_path = 'data/total/'
    fl = 'data/total/total.csv'
    total_file = os.path.exists(fl)
    
    # If there is already 'total.csv', you need to remove it first. If you don't do this, the 'total.csv' file will be appended by new command.
    if total_file:
        os.remove(fl)
        
    file_list = os.listdir(total_path)
    all_files = [file for file  in file_list if file.endswith('.csv')]

    df = pd.DataFrame()

    for i in all_files:
        data = pd.read_csv(total_path + i, index_col=0)
        if len(data.index) == 1:
            pass
        else:
            df = df.append(data)
    df.reset_index(drop = True, inplace=True)
    df.to_csv(total_path + 'total.csv')
    
    # Remove Asean & EU-28 data
    no_asean = df['Reporter'] != 'ASEAN' # code = 975
    no_eu = df['Reporter'] != 'EU-28' # code = 97
    no_other_asia = df['Reporter'] != 'Other Asia, nes' # code = 490
    df = df[no_asean & no_eu & no_other_asia]
    
    if trade == 1:
        df_import = df[df['Trade Flow Code']==1]
        df_import_yearly = df_import.groupby(['Reporter', 'Year']).sum('Netweight (kg)')
        df_import_yearly.reset_index(inplace=True)
        
        plt.figure(figsize=(10,6))

        for country in countries:
            var = 'df_import_yearly_' + country
            var = df_import_yearly[df_import_yearly['Reporter'] == country]
            plt.plot(var['Year'], var['Netweight (kg)'], marker='o')


        plt.legend(countries, loc = "center left", bbox_to_anchor = (1, 0.5), numpoints = 1)
        plt.xlabel('Year', fontsize=14)
        plt.ylabel('Netweight (kg)', fontsize=14)
        plt.title('Yearly Import Trade Amount', fontsize=20)
        plt.savefig('Yearly Import Trade Amount of {}.png'.format(countries), bbox_inches='tight')
        
    elif trade == 2:
        df_export = df[df['Trade Flow Code'] == 2]
        df_export_yearly = df_export.groupby(['Reporter', 'Year']).sum('Netweight (kg)')
        df_export_yearly.reset_index(inplace=True)
        
        plt.figure(figsize=(10,6))
        
        # Plotting the country's result one by one.
        for country in countries:
            var = 'df_export_yearly_' + country
            var = df_export_yearly[df_export_yearly['Reporter'] == country]
            plt.plot(var['Year'], var['Netweight (kg)'], marker='o')


        plt.legend(countries, loc = "center left", bbox_to_anchor = (1, 0.5), numpoints = 1)
        plt.xlabel('Year', fontsize=14)
        plt.ylabel('Netweight (kg)', fontsize=14)
        plt.title('Yearly Export Trade Amount', fontsize=20)
        plt.savefig('Yearly Export Trade Amount of {}.png'.format(countries), bbox_inches='tight')
        
    else:
        print('Select trade number: Import=1, Export=2')
        
    return